<a href="https://colab.research.google.com/github/rameshavinash94/Custom_NER/blob/main/Avinash_Ramesh_Spacy_2_0_Custom_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***NOTE:I have used Spacy 2.0 for training Custom NER, we can also use Spacy 3.0 which makes it even more simpler and easier with added functionalities.***

**IMPORTING LIBRARIES**

In [ ]:
#importing librariess
import spacy
import pandas as pd
import re
from spacy.tokens import DocBin
from tqdm import tqdm
import random
import json

**LOADING THE INITIAL DATASET**

In [ ]:
#load the provided file in a pandas df.
initial_df=pd.read_csv('/content/Summer Internship - Homework Exercise - Summer Internship - Homework Exercise.csv')

***From the Initial dataset, I'm going to seperate the train,test,validation***

In [ ]:
#extract the training,test,validaiton dataset from the initial dataset
training_data = initial_df[initial_df['dataset']=='train'].drop(columns=['dataset'])
testing_data = initial_df[initial_df['dataset']=='test'].drop(columns=['dataset'])
validation_data = initial_df[initial_df['dataset']=='validation'].drop(columns=['dataset'])

In [ ]:
#view the training df
training_data

,transaction_descriptor,store_number
0,DOLRTREE 2257 00022574 ROSWELL,2257
1,AUTOZONE #3547,3547
2,TGI FRIDAYS 1485 0000,1485
3,BUFFALO WILD WINGS 003,3
4,J. CREW #568 0,568
...,...,...
95,DUNKIN #355514,355514
96,THE HOME DEPOT #6828,6828
97,RUE21 #1129 BLUE,1129
98,WM SUPERCENTER #34,34


**here i'm going to perfrom text preprocessing like remove Specialcharacters and ltrip 0 in numbers.**

In [ ]:
#perfrom preprocessing
all_sentences=[]
for x in training_data['transaction_descriptor']:
  tmp1 = re.sub(r"[^a-zA-Z0-9]+", ' ',x) # remove sp characters
  each_sentence=[]
  for t in tmp1.split(' '):
    if t.startswith('0'):
      each_sentence.append(t.lstrip('0'))
    else:
      each_sentence.append(t)
    tmp = re.findall(r"[^\W\d_]+|\d+",' '.join(each_sentence)) # split alphanumeric string to alpha numeric
  all_sentences.append(' '.join(tmp))

In [ ]:
#process store number
final=[]
for x in training_data['store_number']:
  tmp = re.findall(r"[^\W\d_]+|\d+",x) # split alphanumeric string to alpha numeric
  final.append(' '.join(tmp))
training_data['new_store_number']=final

In [ ]:
training_data['preprocssed_data']=all_sentences

In [ ]:
#view the df
training_data

,transaction_descriptor,store_number,new_store_number,preprocssed_data
0,DOLRTREE 2257 00022574 ROSWELL,2257,2257,DOLRTREE 2257 22574 ROSWELL
1,AUTOZONE #3547,3547,3547,AUTOZONE 3547
2,TGI FRIDAYS 1485 0000,1485,1485,TGI FRIDAYS 1485
3,BUFFALO WILD WINGS 003,3,3,BUFFALO WILD WINGS 3
4,J. CREW #568 0,568,568,J CREW 568
...,...,...,...,...
95,DUNKIN #355514,355514,355514,DUNKIN 355514
96,THE HOME DEPOT #6828,6828,6828,THE HOME DEPOT 6828
97,RUE21 #1129 BLUE,1129,1129,RUE 21 1129 BLUE
98,WM SUPERCENTER #34,34,34,WM SUPERCENTER 34


***Now, i going to create a entity index of store number in the transaction description to train my model***

In [ ]:
#create entity index
entitiy_index = [(re.search(y,x).start(),re.search(y,x).end()) for x,y, in zip(training_data.preprocssed_data,training_data.new_store_number)]

In [ ]:
#add it to new column for easier visualization
training_data['entity_index'] = entitiy_index

In [ ]:
#view th df
training_data

,transaction_descriptor,store_number,new_store_number,preprocssed_data,entity_index
0,DOLRTREE 2257 00022574 ROSWELL,2257,2257,DOLRTREE 2257 22574 ROSWELL,"(9, 13)"
1,AUTOZONE #3547,3547,3547,AUTOZONE 3547,"(9, 13)"
2,TGI FRIDAYS 1485 0000,1485,1485,TGI FRIDAYS 1485,"(12, 16)"
3,BUFFALO WILD WINGS 003,3,3,BUFFALO WILD WINGS 3,"(19, 20)"
4,J. CREW #568 0,568,568,J CREW 568,"(7, 10)"
...,...,...,...,...,...
95,DUNKIN #355514,355514,355514,DUNKIN 355514,"(7, 13)"
96,THE HOME DEPOT #6828,6828,6828,THE HOME DEPOT 6828,"(15, 19)"
97,RUE21 #1129 BLUE,1129,1129,RUE 21 1129 BLUE,"(7, 11)"
98,WM SUPERCENTER #34,34,34,WM SUPERCENTER 34,"(15, 17)"


***Conver Df into required format for training***

In [ ]:
#create the training data in the required format for processing
train_data=[]
for index, row in training_data.iterrows():
    train_data.append((row['preprocssed_data'],{'entities':[[row['entity_index'][0],row['entity_index'][1],'store_number']]}))

In [ ]:
train_data

[('DOLRTREE 2257 22574 ROSWELL', {'entities': [[9, 13, 'store_number']]}),
 ('AUTOZONE 3547', {'entities': [[9, 13, 'store_number']]}),
 ('TGI FRIDAYS 1485', {'entities': [[12, 16, 'store_number']]}),
 ('BUFFALO WILD WINGS 3', {'entities': [[19, 20, 'store_number']]}),
 ('J CREW 568', {'entities': [[7, 10, 'store_number']]}),
 ('KRISPY KREME 40 GREENVILLE SC', {'entities': [[13, 15, 'store_number']]}),
 ('FIVE GUYS MN 1847 ECOM 612 339 9733 MN',
  {'entities': [[13, 17, 'store_number']]}),
 ('CASEYS GEN STORE 2650', {'entities': [[17, 21, 'store_number']]}),
 ('HUDDLE HOUSE 535', {'entities': [[13, 16, 'store_number']]}),
 ('JAMBA JUICE 1305', {'entities': [[12, 16, 'store_number']]}),
 ('ANN TAYLOR FACTORY 2202', {'entities': [[19, 23, 'store_number']]}),
 ('Subway 26824', {'entities': [[7, 12, 'store_number']]}),
 ('MARSHALLS 688', {'entities': [[10, 13, 'store_number']]}),
 ('OREILLY AUTO 4681', {'entities': [[13, 17, 'store_number']]}),
 ('TA 227 BARSTOW REST', {'entities': [[3, 6,

In [ ]:
#conver it to json
convert_json = json.dumps(train_data)

In [ ]:
#load the trained json in a variable
train_data= json.loads(convert_json)

In [ ]:
#view the json
train_data

[['DOLRTREE 2257 22574 ROSWELL', {'entities': [[9, 13, 'store_number']]}],
 ['AUTOZONE 3547', {'entities': [[9, 13, 'store_number']]}],
 ['TGI FRIDAYS 1485', {'entities': [[12, 16, 'store_number']]}],
 ['BUFFALO WILD WINGS 3', {'entities': [[19, 20, 'store_number']]}],
 ['J CREW 568', {'entities': [[7, 10, 'store_number']]}],
 ['KRISPY KREME 40 GREENVILLE SC', {'entities': [[13, 15, 'store_number']]}],
 ['FIVE GUYS MN 1847 ECOM 612 339 9733 MN',
  {'entities': [[13, 17, 'store_number']]}],
 ['CASEYS GEN STORE 2650', {'entities': [[17, 21, 'store_number']]}],
 ['HUDDLE HOUSE 535', {'entities': [[13, 16, 'store_number']]}],
 ['JAMBA JUICE 1305', {'entities': [[12, 16, 'store_number']]}],
 ['ANN TAYLOR FACTORY 2202', {'entities': [[19, 23, 'store_number']]}],
 ['Subway 26824', {'entities': [[7, 12, 'store_number']]}],
 ['MARSHALLS 688', {'entities': [[10, 13, 'store_number']]}],
 ['OREILLY AUTO 4681', {'entities': [[13, 17, 'store_number']]}],
 ['TA 227 BARSTOW REST', {'entities': [[3, 6,

***SPACY TRAINING FUNCTION TO CREATE A CUSTOM NER MODEL TO TRAIN***

In [ ]:
def train_spacy_nlp(train_data,iterations):
  nlp=spacy.blank('en')
  ner = nlp.create_pipe('ner')
  ner.add_label('store_number')
  nlp.add_pipe(ner, name='store_number_ner')
  disable_pipe = [pipe for pipe in nlp.pipe_names if pipe != 'store_number_ner']
  with nlp.disable_pipes(*disable_pipe):
    optimizer=nlp.begin_training()
    for iter in range(iterations):
      print('starting interation {iter}'.format(iter=iter))
      random.shuffle(train_data)
      losses={}
      for text, annotations in train_data:
        nlp.update([text],[annotations],drop=0.3,sgd=optimizer,losses=losses)
      
      print(losses)
  return (nlp)

In [ ]:
#call the funciton and train the custom NER for store number
trained = train_spacy_nlp(train_data,20)

starting interation 0
{'ner': 91.37464968123959}
starting interation 1
{'ner': 22.420725569011832}
starting interation 2
{'ner': 8.094795953362848}
starting interation 3
{'ner': 4.111727390238749}
starting interation 4
{'ner': 4.26505134056762}
starting interation 5
{'ner': 3.9155384014447714}
starting interation 6
{'ner': 1.9992156846889007}
starting interation 7
{'ner': 9.201255092213117}
starting interation 8
{'ner': 4.02130037531638}
starting interation 9
{'ner': 1.9384407472284264}
starting interation 10
{'ner': 3.243326126911465}
starting interation 11
{'ner': 3.6133959602587407}
starting interation 12
{'ner': 2.535466051159143}
starting interation 13
{'ner': 2.0707169485008468}
starting interation 14
{'ner': 2.9111114713493773}
starting interation 15
{'ner': 1.9523576018264837}
starting interation 16
{'ner': 5.4864394795793885}
starting interation 17
{'ner': 3.1761110046752425}
starting interation 18
{'ner': 2.37657013590051}
starting interation 19
{'ner': 0.0007196199999577954}

***Perform preprocessing prior passing to our Custom NER model***

In [ ]:
#this function is to perfrom porecessing prior passing to our NER model
def preprocessing(sentence):
    each_sentence=[]
    tmp1 = re.sub(r"[^a-zA-Z0-9]+", ' ', sentence) # remove sp characters
    for t in tmp1.split(' '):
      if t.startswith('0'):
        each_sentence.append(t.lstrip('0'))
      else:
        each_sentence.append(t)
    tmp = re.findall(r"[^\W\d_]+|\d+",' '.join(each_sentence)) 
    return ' '.join(tmp)

**RUNNING MY TRAINED MODEL ON TESTING DATA**

In [ ]:
final=[]
for x in testing_data['transaction_descriptor']:
  preprocess = preprocessing(x)
  doc=trained(preprocess)
  if doc.ents:
    spacy.displacy.render(doc, style="ent", jupyter=True)

***Store the results in the testing dataframe and comare actual with extracted value***

In [ ]:
#store in testing_df and visualize and compare
final=[]
for x in testing_data['transaction_descriptor']:
  preprocess = preprocessing(x)
  doc=trained(preprocess)
  if doc.ents:
    tmp=[]
    for x in doc.ents:
      tmp.append(x.text.replace(' ',''))
    final.append(' | '.join(tmp))
testing_data['found_store_number_entities']=final

In [ ]:
testing_data['Correct_prediction'] = (testing_data['found_store_number_entities'] == testing_data['store_number']).astype(str)

In [ ]:
testing_data

,transaction_descriptor,store_number,found_store_number_entities,Correct_prediction
200,IN-N-OUT BURGER #242,242,242,True
201,BP#9442088LIBERTYVILLE B,9442088,9442088,True
202,JCPENNEY 1419,1419,1419,True
203,ROSS STORES #1019,1019,1019,True
204,WM SUPERCENTER #38,38,38,True
...,...,...,...,...
295,MCDONALD'S F2151,F2151,F2151,True
296,NST BEST BUY #1403 332411,1403,1403,True
297,CVS/PHARMACY #06689,6689,6689,True
298,BANANA REPUBLIC #8109,8109,8109,True


In [ ]:
testing_data.to_csv('test_data_entity_extraction.csv')

In [ ]:
#print wrong predictions
testing_data[testing_data['Correct_prediction'] == 'False']

,transaction_descriptor,store_number,found_store_number_entities,Correct_prediction
222,BP#8644346ES #30 B96,8644346,8644346 | 96,False
223,NNT POLO/RL WRENTHA130571,13057,130571,False
228,"WINN DIXIE #2454 SEFFNER, FL (1033)",2454,2454 | 1033,False
231,NNT SEARS HOMETOWN 862751,8627,862751,False
253,MCDONALD'S F2383 972-231-3337 TX,F2383,F2383 | 3337,False
264,NNT FAMOUS FOOTWEAR001261,1261,001261,False
290,PAPA MURPHY'S UT044 OLO,UT044,044,False
292,SUBWAY 00032128,3212,32128,False


In [ ]:
#Percentage of Correct Predictions
correct_predictions = len(testing_data[testing_data['Correct_prediction'] == 'True'])
total_entries = len(testing_data)
print((correct_predictions/total_entries)*100)

92.0


**RUNNING ON VALIDATION DATA**

In [ ]:
for x in validation_data['transaction_descriptor']:
  preprocess = preprocessing(x)
  doc=trained(preprocess)
  if doc.ents:
    spacy.displacy.render(doc, style="ent", jupyter=True)

***Store the results in the validaiton dataframe and compare actual with extracted value***

In [ ]:
#store in testing_df and visualize and compare
final=[]
for x in validation_data['transaction_descriptor']:
  preprocess = preprocessing(x)
  doc=trained(preprocess)
  if doc.ents:
    tmp=[]
    for x in doc.ents:
      tmp.append(x.text.replace(' ',''))
    final.append('|'.join(tmp))
  else:
    final.append(None)
validation_data['found_store_number_entities']=final

In [ ]:
validation_data['Correct_prediction'] = (validation_data['found_store_number_entities'] == validation_data['store_number']).astype(str)

In [ ]:
validation_data.to_csv('validation_data_entity_extraction.csv')

In [ ]:
#print wrong predictions
validation_data[validation_data['Correct_prediction'] == 'False']

,transaction_descriptor,store_number,found_store_number_entities,Correct_prediction
119,PIER 1 IMPORTS00008060,806,1,False
120,"LOWES #02282* BRANDON, FL (7679)",2282,2282|7679,False
129,"WENDYS #514 TAMPA, FL (7271)",514,514|7271,False
134,NNT BURLNGTON STORE001491,1491,001491,False
146,NNT SEARS HOMETOWN 000415,4,415,False
150,KUM & GO #927,927,GO927,False
152,PAPA MURPHY'S CA142,CA142,142,False
155,KFC L765004 66850041,L765004,765004,False
167,SUBWAY 00040055,4005,40055,False
185,SIX 02 46221 SIX 02 46221,46221,2,False


In [ ]:
#Percentage of Correct Predictions
correct_predictions = len(validation_data[validation_data['Correct_prediction'] == 'True'])
total_entries = len(validation_data)
print((correct_predictions/total_entries)*100)

90.0


*From the above, we can say that, our model performed well on almost most occasion and it was able to find certain entites which are not expected. We can further retrain/tune our model with our testing data and then check validation based on business needs/insights.*

## **COMBINE TRAIN, TEST AND TRAIN THE MODEL AND THEN CHECK VALIDATION ACCURACY.**

In [ ]:
combined = initial_df[(initial_df['dataset']=='train') | (initial_df['dataset']=='test')].drop(columns=['dataset'])

**here i'm going to perfrom text preprocessing like remove Specialcharacters and ltrip 0 in numbers.**

In [ ]:
#perfrom preprocessing
all_sentences=[]
for x in combined['transaction_descriptor']:
  tmp1 = re.sub(r"[^a-zA-Z0-9]+", ' ',x) # remove sp characters
  each_sentence=[]
  for t in tmp1.split(' '):
    if t.startswith('0'):
      each_sentence.append(t.lstrip('0'))
    else:
      each_sentence.append(t)
    tmp = re.findall(r"[^\W\d_]+|\d+",' '.join(each_sentence)) # split alphanumeric string to alpha numeric
  all_sentences.append(' '.join(tmp))

In [ ]:
#process store number
final=[]
for x in combined['store_number']:
  tmp = re.findall(r"[^\W\d_]+|\d+",x) # split alphanumeric string to alpha numeric
  final.append(' '.join(tmp))
combined['new_store_number']=final

In [ ]:
combined['preprocssed_data']=all_sentences

***Now, i going to create a entity index of store number in the transaction description to train my model***

In [ ]:
#create entity index
entitiy_index = [(re.search(y,x).start(),re.search(y,x).end()) for x,y, in zip(combined.preprocssed_data,combined.new_store_number)]

In [ ]:
#add it to new column for easier visualization
combined['entity_index'] = entitiy_index

***Conver Df into required format for training***

In [ ]:
#create the training data in the required format for processing
combined_data=[]
for index, row in combined.iterrows():
    combined_data.append((row['preprocssed_data'],{'entities':[[row['entity_index'][0],row['entity_index'][1],'store_number']]}))

In [ ]:
combined_data

[('DOLRTREE 2257 22574 ROSWELL', {'entities': [[9, 13, 'store_number']]}),
 ('AUTOZONE 3547', {'entities': [[9, 13, 'store_number']]}),
 ('TGI FRIDAYS 1485', {'entities': [[12, 16, 'store_number']]}),
 ('BUFFALO WILD WINGS 3', {'entities': [[19, 20, 'store_number']]}),
 ('J CREW 568', {'entities': [[7, 10, 'store_number']]}),
 ('KRISPY KREME 40 GREENVILLE SC', {'entities': [[13, 15, 'store_number']]}),
 ('FIVE GUYS MN 1847 ECOM 612 339 9733 MN',
  {'entities': [[13, 17, 'store_number']]}),
 ('CASEYS GEN STORE 2650', {'entities': [[17, 21, 'store_number']]}),
 ('HUDDLE HOUSE 535', {'entities': [[13, 16, 'store_number']]}),
 ('JAMBA JUICE 1305', {'entities': [[12, 16, 'store_number']]}),
 ('ANN TAYLOR FACTORY 2202', {'entities': [[19, 23, 'store_number']]}),
 ('Subway 26824', {'entities': [[7, 12, 'store_number']]}),
 ('MARSHALLS 688', {'entities': [[10, 13, 'store_number']]}),
 ('OREILLY AUTO 4681', {'entities': [[13, 17, 'store_number']]}),
 ('TA 227 BARSTOW REST', {'entities': [[3, 6,

In [ ]:
#conver it to json
convert_json = json.dumps(combined_data)

In [ ]:
#load the trained json in a variable
combined_train_data= json.loads(convert_json)

In [ ]:
#view the json
combined_train_data

[['DOLRTREE 2257 22574 ROSWELL', {'entities': [[9, 13, 'store_number']]}],
 ['AUTOZONE 3547', {'entities': [[9, 13, 'store_number']]}],
 ['TGI FRIDAYS 1485', {'entities': [[12, 16, 'store_number']]}],
 ['BUFFALO WILD WINGS 3', {'entities': [[19, 20, 'store_number']]}],
 ['J CREW 568', {'entities': [[7, 10, 'store_number']]}],
 ['KRISPY KREME 40 GREENVILLE SC', {'entities': [[13, 15, 'store_number']]}],
 ['FIVE GUYS MN 1847 ECOM 612 339 9733 MN',
  {'entities': [[13, 17, 'store_number']]}],
 ['CASEYS GEN STORE 2650', {'entities': [[17, 21, 'store_number']]}],
 ['HUDDLE HOUSE 535', {'entities': [[13, 16, 'store_number']]}],
 ['JAMBA JUICE 1305', {'entities': [[12, 16, 'store_number']]}],
 ['ANN TAYLOR FACTORY 2202', {'entities': [[19, 23, 'store_number']]}],
 ['Subway 26824', {'entities': [[7, 12, 'store_number']]}],
 ['MARSHALLS 688', {'entities': [[10, 13, 'store_number']]}],
 ['OREILLY AUTO 4681', {'entities': [[13, 17, 'store_number']]}],
 ['TA 227 BARSTOW REST', {'entities': [[3, 6,

***SPACY TRAINING FUNCTION TO CREATE A CUSTOM NER MODEL TO TRAIN***

In [ ]:
#call the funciton and train the custom NER for store number
combined_trained = train_spacy_nlp(combined_train_data,20)

starting interation 0
{'ner': 116.02360421550752}
starting interation 1
{'ner': 21.797754902850848}
starting interation 2
{'ner': 6.8268181667110115}
starting interation 3
{'ner': 7.904766165000587}
starting interation 4
{'ner': 4.191159701609829}
starting interation 5
{'ner': 3.284526966723193}
starting interation 6
{'ner': 11.285370838320636}
starting interation 7
{'ner': 7.692099147195223}
starting interation 8
{'ner': 5.273457786587894}
starting interation 9
{'ner': 1.7860772504000453}
starting interation 10
{'ner': 11.179214475356318}
starting interation 11
{'ner': 2.7335113303963054}
starting interation 12
{'ner': 2.00006822599901}
starting interation 13
{'ner': 2.1299204199980837}
starting interation 14
{'ner': 2.210362667751158}
starting interation 15
{'ner': 13.25835262158291}
starting interation 16
{'ner': 1.4925938600759507}
starting interation 17
{'ner': 8.277658996975843e-11}
starting interation 18
{'ner': 1.7751432990483403e-10}
starting interation 19
{'ner': 3.4646011960

**RUNNING ON VALIDATION DATA**

In [ ]:
for x in validation_data['transaction_descriptor']:
  preprocess = preprocessing(x)
  doc=combined_trained(preprocess)
  if doc.ents:
    spacy.displacy.render(doc, style="ent", jupyter=True)

***Store the results in the validaiton dataframe and compare actual with extracted value***

In [ ]:
#store in testing_df and visualize and compare
final=[]
for x in validation_data['transaction_descriptor']:
  preprocess = preprocessing(x)
  doc=combined_trained(preprocess)
  if doc.ents:
    tmp=[]
    for x in doc.ents:
      tmp.append(x.text.replace(' ',''))
    final.append('|'.join(tmp))
  else:
    final.append(None)
validation_data['found_store_number_entities']=final

In [ ]:
validation_data['Correct_prediction'] = (validation_data['found_store_number_entities'] == validation_data['store_number']).astype(str)

In [ ]:
validation_data.to_csv('validation_data_entity_extraction_final.csv')

In [ ]:
#print wrong predictions
validation_data[validation_data['Correct_prediction'] == 'False']

,transaction_descriptor,store_number,found_store_number_entities,Correct_prediction
119,PIER 1 IMPORTS00008060,806,1,False
134,NNT BURLNGTON STORE001491,1491,001491,False
146,NNT SEARS HOMETOWN 000415,4,415,False
150,KUM & GO #927,927,GO927,False
152,PAPA MURPHY'S CA142,CA142,142,False
167,SUBWAY 00040055,4005,40055,False
185,SIX 02 46221 SIX 02 46221,46221,2,False


In [ ]:
#Percentage of Correct Predictions
correct_predictions = len(validation_data[validation_data['Correct_prediction'] == 'True'])
total_entries = len(validation_data)
print((correct_predictions/total_entries)*100)

93.0


**CONVER TO HTML**

In [ ]:
!jupyter nbconvert --to html /content/Avinash_Ramesh_Spacy_2_0_Custom_NER.ipynb

[NbConvertApp] Converting notebook /content/Avinash_Ramesh_Spacy_2_0_Custom_NER.ipynb to html
[NbConvertApp] Writing 606140 bytes to /content/Avinash_Ramesh_Spacy_2_0_Custom_NER.html
